In [ ]:
#importing Libraries required to run the model..
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
if int(os.environ.get("MODERN_PANDAS_EPUB", 0)):
    import prep # noqa
from sklearn.metrics import mutual_info_score

pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 10
sns.set(style='ticks', context='talk')
# Read csv in a Spark
#Gender = pd.read_csv(GenderFilePath,sep='^') 
#Gender.info()

import re
import numpy as np
import pandas as pd
from pprint import pprint
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import re
from urllib.parse import urlparse
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import nltk
nltk.download('stopwords')

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Read csv in a Spark
GenderFilePath = "/usr/local/bin/Gender Prediction/Data/genderprediction.txt"

import pandas as pd
Gender = pd.read_csv(GenderFilePath, delimiter='\t', names=["UserId", "Gender", "Age", "Language", "description", "long_description", "URL", "DateTime"])

Gender.head(1000)

In [ ]:
#Copy the dataframe
df= Gender.copy()

In [ ]:
# Load Stop words
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
def Datapreprocess(df):
    
    # Filling long_description missing values with description column..
    df['long_description'].fillna(df['description'])
#droping missing values....
    del df['description']
    df.dropna(subset=['UserId'],inplace=True)
    df.dropna(subset=['Gender'],inplace=True)
    df.dropna(subset=['long_description'],inplace=True)
    df.dropna(subset=['DateTime'],inplace=True)

    df['Create Time'] = pd.to_datetime(df['DateTime'])

    # Category Based Features 
    df['URL'].str.strip()
    df['domain'] = df['URL'].str.split('//').str[-1].str.split('/').str[0]
    df['Count'] = df['URL'].str.split('//').str[-1].str.split('/').str.len()
    mask = df['Count'] > 2

    df['Categ'] = np.where(mask,df['URL'].str.split('//').str[-1].str.split('/').str[1], 'No Category')
    df['Subcateg'] = np.where(mask, df['URL'].str.split('//').str[-1].str.split('/').str[2], 'No Subcategory')

    df['Categ'] = df.Categ.astype('category')
    df['Subcateg']= df.Subcateg.astype('category')
    df['domain'] = df.domain.astype('category')
    df1=df[mask]
    # Time n Squential Based
    df1['Gender'] = df1['Gender'].astype('category')
    df1["Create Time"]= pd.to_datetime(df1['Create Time'])
    df1['time_hour'] = df1['Create Time'].apply(lambda x: x.hour)
    df1.Gender.replace(['M', 'F'], [0, 1], inplace=True)

    df2 = df1[df1['long_description'].notna()]
    def sent_to_words(sentences):
        for sent in sentences:
            sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
            sent = re.sub('\s+', ' ', sent)  # remove newline chars
            sent = re.sub("\'", "", sent)  # remove single quotes
            sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
            yield(sent)  
    # Convert to list
    data_ld = df2.long_description.values.tolist()

    data_words = list(sent_to_words(data_ld))
     #Define functions for stopwords, bigrams, trigrams and lemmatization
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    #For Gibbs sampling using MAllet package...
    mallet_path = '/usr/local/bin//mallet-2.0.8/bin/mallet' # update this path
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=20, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)

    def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ld):
        # Init output
        sent_topics_df = pd.DataFrame()

        # Get main topic in each document
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
        return(sent_topics_df)

    # Compute Coherence Score
    def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
        """
        Compute c_v coherence for various number of topics
    Parameters:
        ----------
        dictionary : Gensim dictionary
        corpus : Gensim corpus
        texts : List of input texts
        limit : Max num of topics
     Returns:
        -------
        model_list : List of LDA topic models
        coherence_values : Coherence values corresponding to the LDA model with respective number of topics
        """
        coherence_values = []
        model_list = []
        for num_topics in range(start, limit, step):
            model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(coherencemodel.get_coherence())

        return model_list, coherence_values
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)
    # Select the model and print the topics
    optimal_model = model_list[3]
    model_topics = optimal_model.show_topics(formatted=False)

    df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data_ld )

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'long_description']

    df3 = pd.merge(df1, df_dominant_topic, on='long_description')

    params = ["UserId", "Gender", "Create Time","Categ","time_hour", "URL", "Subcateg", "Topic_Perc_Contrib", "Age", "domain"]

    sqFeatures = df3[params]

    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    cat_columns = [ "Gender","Categ","time_hour", "Subcateg", "Age", "domain"]
    sqFeatures[cat_columns].apply(lambda col: col.astype('category'))
    # add it to our new dataframe
    label_encoders = {}
    for col in cat_columns:
        print("Encoding {}".format(col))
        new_le = LabelEncoder()
        sqFeatures[col] = pd.factorize(sqFeatures[col])[0]
        sqFeatures[col] = new_le.fit_transform(sqFeatures[col])
        label_encoders[col] = new_le  

    sqFeatures = sqFeatures.groupby('UserId')[[ "Gender", "Create Time","Categ","time_hour", "URL", "Subcateg", "Topic_Perc_Contrib", "Age", "domain"]].agg(lambda x: x.tolist())


    # Calculate Mutual Info..
    from sklearn.metrics import mutual_info_score
    def calc_MI(x, y, bins):
        c_xy = np.histogram2d(x, y, bins)[0]
        mi = mutual_info_score(None, None, contingency=c_xy)
        return mi

    sqFeatures['mutualInfo'] = sqFeatures.apply(lambda row: calc_MI(row['Gender'], row['Categ'], 30), axis=1)
    sqFeatures['mutualInfo_sub'] = sqFeatures.apply(lambda row: calc_MI(row['Gender'], row['Subcateg'], 30), axis=1)
    sqFeatures.loc[:, 'Gender'] = sqFeatures.Gender.map(lambda x: x[0])
    sqFeatures.loc[:, 'Age'] = sqFeatures.Age.map(lambda x: x[0])
    label1 = sqFeatures['Gender']

    sqFeatures['domain'] = sqFeatures['domain'].apply(lambda x: np.array(x))
    sqFeatures['Categ'] = sqFeatures['Categ'].apply(lambda x: np.array(x))
    sqFeatures['time_hour'] = sqFeatures['time_hour'].apply(lambda x: np.array(x))
    sqFeatures['Subcateg'] = sqFeatures['Subcateg'].apply(lambda x: np.array(x))
    sqFeatures['Topic_Perc_Contrib'] = sqFeatures['Topic_Perc_Contrib'].apply(lambda x: np.array(x))

    sqFeatures['mutualInfo'] = sqFeatures['mutualInfo'].apply(lambda x: np.array(x))
    sqFeatures['mutualInfo_sub'] = sqFeatures['mutualInfo_sub'].apply(lambda x: np.array(x))

    max_len = sqFeatures['domain'].apply(lambda x: len(x)).max()
    print(max_len)

    sqFeatures['Categ'] = sqFeatures['Categ'].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=0))
    sqFeatures['domain'] = sqFeatures['domain'].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=0))
    sqFeatures['mutualInfo_re'] =  pd.Series(np.pad(np.array(sqFeatures['mutualInfo']) .reshape(-1, 1), ((0, 0), (0, max_len-1)), 'constant').tolist(), index = sqFeatures.index)
    sqFeatures['time_hour'] = sqFeatures['time_hour'].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=0))        
    sqFeatures['Subcateg'] = sqFeatures['Subcateg'].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=0))
    #sqFeatures['mutualInfo_sub'] = sqFeatures['mutualInfo_sub'].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=0))
    sqFeatures['Topic_Perc_Contrib'] = sqFeatures['Topic_Perc_Contrib'].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=0))
    Features = ["Categ","time_hour", "Subcateg", "Topic_Perc_Contrib", "domain","mutualInfo_re"]
    sqFeatures['combined'] = sqFeatures[Features].values.tolist()
    DF = sqFeatures[['Age','Gender','combined']]
    return DF

In [ ]:
DF = Datapreprocess(df)

In [ ]:
map(lambda x : x,DF['combined'].iloc[0:1])

series = DF['combined'].apply(lambda x : np.array(x)).as_matrix().reshape(-1,1)

features = np.apply_along_axis(lambda x : x[0], 1, series)

y = DF['Gender'].values.reshape(-1,1)

# Gender Prediction

In [ ]:
nsamples, nx, ny = features.shape

In [ ]:
x = features.reshape(nsamples,nx*ny)

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = np.random.seed
scoring = 'accuracy'
validation_size = 0.20

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, np.ravel(y), test_size = 0.2
                                                    ,random_state = 0)
print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

In [ ]:
# Spot-Check Algorithms
models = []
models.append(( ' LR ' , LogisticRegression()))
models.append(( ' LDA ' , LinearDiscriminantAnalysis()))
models.append(( ' KNN ' , KNeighborsClassifier()))
models.append(( ' CART ' , DecisionTreeClassifier()))
models.append(( ' NB ' , GaussianNB()))
models.append(( ' SVM ' , SVC()))

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',
LogisticRegression())])))
#pipelines.append(('ScaledLDA', Pipeline([('Scaler', StandardScaler()),('LDA',
#LinearDiscriminantAnalysis())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN',
KNeighborsClassifier())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART',
DecisionTreeClassifier())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB',
GaussianNB())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVC())])))
results = []
names = []

for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:

# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(( ' ScaledLR ' , Pipeline([( ' Scaler ' , StandardScaler()),( ' LR ' ,
LogisticRegression())])))
pipelines.append(( ' ScaledLDA ' , Pipeline([( ' Scaler ' , StandardScaler()),( ' LDA ' ,
LinearDiscriminantAnalysis())])))

In [ ]:
pipelines.append(( ' ScaledKNN ' , Pipeline([( ' Scaler ' , StandardScaler()),( ' KNN ' ,
KNeighborsClassifier())])))
pipelines.append(( ' ScaledCART ' , Pipeline([( ' Scaler ' , StandardScaler()),( ' CART ' ,
DecisionTreeClassifier())])))
pipelines.append(( ' ScaledNB ' , Pipeline([( ' Scaler ' , StandardScaler()),( ' NB ' ,
GaussianNB())])))
pipelines.append(( ' ScaledSVM ' , Pipeline([( ' Scaler ' , StandardScaler()),( ' SVM ' , SVC())])))
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle( ' Scaled Algorithm Comparison ' )
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
# Tune scaled KNN
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
neighbors = [1,3,5,7,9,11,13,15,17,19,21]
param_grid = dict(n_neighbors=neighbors)
model = KNeighborsClassifier()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
# Tune scaled SVM
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# ensembles
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
ensembles.append(('RF', RandomForestClassifier()))
ensembles.append(('ET', ExtraTreesClassifier()))
results = []
names = []
for name, model in ensembles:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

In [ ]:

# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:

# Finalize Model

# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = SVC(C=1.5)
model.fit(rescaledX, Y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_test)
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))
print(classification_report(Y_test, predictions))


# Age Prediction

In [ ]:
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error


In [ ]:
y1  = DF['Age'].values.reshape(-1,1)

In [ ]:
Scoring = 'neg_mean_squared_error'
seed = 7

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, np.ravel(y1), test_size = 0.2
                                                    ,random_state = 0)
print (Xtrain.shape, Ytrain.shape)
print (Xtest.shape, Ytest.shape)

In [ ]:
# Spot Check Algorithms
algos = []
algos.append(('LR', LinearRegression()))
algos.append(('LASSO', Lasso()))
algos.append(('EN', ElasticNet()))
algos.append(('KNN', KNeighborsRegressor()))
algos.append(('CART', DecisionTreeRegressor()))
algos.append(('SVR', SVR()))

In [ ]:
# evaluate each model in turn
out = []
Names = []
for Name, algo in algos:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, Xtrain, Ytrain, cv=kfold, scoring=Scoring)
	out.append(cv_results)
	Names.append(Name)
	Msg = "%s: %f (%f)" % (Name, cv_results.mean(), cv_results.std())
	print(Msg)



In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(out)
ax.set_xticklabels(Names)
pyplot.show()

In [ ]:
# Standardize the dataset
Pipelines = []
Pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
Pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
Pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
Pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
Pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
Pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())])))
out = []
Names = []
for Name, algos in Pipelines:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, Xtrain, Ytrain, cv=kfold, scoring=Scoring)
	out.append(cv_results)
	Names.append(Name)
	msg = "%s: %f (%f)" % (Name, cv_results.mean(), cv_results.std())
	print(msg)

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(out)
ax.set_xticklabels(Names)
pyplot.show()


In [ ]:
# KNN Algorithm tuning
scaler = StandardScaler().fit(Xtrain)
rescaleX = scaler.transform(Xtrain)
k_values = np.array([1,3,5,7,9,11,13,15,17,19,21])
param_grid = dict(n_neighbors=k_values)
model = KNeighborsRegressor()
kfold = KFold(n_splits=num_folds, random_state=seed)
Grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=Scoring, cv=kfold)
Grid_result = grid.fit(rescaleX, Ytrain)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
# ensembles
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor())])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor())])))
out = []

In [ ]:
for Name, algos in ensembles:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(algos, Xtrain, Ytrain, cv=kfold, scoring=Scoring)
	out.append(cv_results)
	Names.append(Name)
	msg = "%s: %f (%f)" % (Name, cv_results.mean(), cv_results.std())
	print(msg)

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Scaled Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(out)
ax.set_xticklabels(Names)
pyplot.show()

In [ ]:
# Tune scaled GBM
scaler = StandardScaler().fit(Xtrain)
rescaleX = scaler.transform(Xtrain)
param_grid = dict(n_estimators=np.array([50,100,150,200,250,300,350,400]))
model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=Scoring, cv=kfold)
grid_result = grid.fit(rescaleX, Ytrain)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Make predictions on validation dataset

# prepare the model
scaler = StandardScaler().fit(Xtrain)
rescaleX = scaler.transform(Xtrain)
model = GradientBoostingRegressor(random_state=seed, n_estimators=400)
model.fit(rescaleX, Ytrain)

In [ ]:
# transform the validation dataset
rescaledValidationX = scaler.transform(Xtest)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(Ytest, predictions))


References:
1. https://machinelearningmastery.com/
2. https://www.researchgate.net/publication/278399485_Gender_Prediction_Using_Browsing_History